# Expressions

In [ ]:
import hail as hl
hl.init()

# Eager Evaluation

Python and R use **eager** evaulation.

When you enter an expression, the result is computed immediately and stored.

In [ ]:
1 + 2

# Lazy Evaluation

Eager evaluation won't work on datasets that won't fit in memory.

Consider the UK Biobank BGEN file, which is ~2TB but decompresses to >100TB in memory.

In order to process datasets of this size, Hail uses lazy evaluation.

When you enter an expression, Hail doesn't execute the expression immediately: it simply records what you asked to do.

Hail `Expression` objects have an (unsupported) method to print the expression they record.

In [ ]:
one = hl.int32(1)
three = one + 2
three._ast.to_hql()

Hail evaluates an expression only when it must, for example:

 - when performing an aggregation,
 - when calling `take`, `collect` or `show`,
 - when exporting or writing to disk.

Hail evaluates expressions by streaming to accomodate very large datasets.

You can expressions with no index by calling `value`. The `show` method also prints the type.

In [ ]:
three.value

In [ ]:
three.show()

# Indices

Expressions carry another piece of information: indices.  Indices record the `Table` or `MatrixTable` the expression refers to, and what axes the `MatrixTable` varies over.

Let's see some examples.

Let's load the 1000 genomes dataset again.

In [ ]:
hl.utils.get_1kg('data/')

In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')
mt

Let's add a global field.

In [ ]:
mt = mt.annotate_globals(dataset = '1kg')

And examine some fields.

In [ ]:
mt.dataset.describe()

In [ ]:
mt.locus.describe()

In [ ]:
mt.s.describe()

In [ ]:
mt.GT.describe()

Expressions with an index (like `locus`, `s`, and `GT` above) have no one `value`, because these values vary across rows or columns of the matrix.

You can get the value of an expression with no indices with the `value` property.

In [ ]:
mt.dataset.value

# `show`, `take`, and `collect`

Although expressions with indices have no `value`, you can use `show` to print the first few values, or `take` and `collect` to localize value to Python.

In [ ]:
mt.s.show()

In [ ]:
mt.s.take(5)

You can `collect` an expression to localize all values, like getting a list of all sample IDs of a dataset.

But be careful -- don't `collect` more data than can fit in memory!

In [ ]:
all_sample_ids = mt.s.collect()
all_sample_ids[:5]

# Learning more

Hail has a suite of of [functions](https://hail.is/docs/devel/functions/index.html) to transform and build expressions.

You should also look at the documentation for the [expressions](https://hail.is/docs/devel/expressions.html) themselves.